## Отчет по лабораторной работе № 3

Тарасенко Александр, 4 курс 12 группа


#### Задание 1. Для заданного M (табл. 4) определить значения a и b, которые позволяют построить эллиптическую группу EM(a, b).


Задание параметров: 

M = 59 

a = 1, b = 0

#### Задание 2. Для найденных в задании 1 параметров сгенерировать все элементы эллиптической группы EM(a, b).

In [1]:
import hashlib
from sympy import randprime
import random
import numpy as np

In [2]:
def modular_sqrt(a, p):
    if legendre_symbol(a, p) != 1:
        return 0
    elif a == 0:
        return 0
    elif p == 2:
        return 0
    elif p % 4 == 3:
        return pow(a, (p + 1) // 4, p)

    s = p - 1
    e = 0
    while s % 2 == 0:
        s = s // 2
        e += 1

    n = 2
    while legendre_symbol(n, p) != -1:
        n += 1

    x = pow(a, (s + 1) // 2, p)
    b = pow(a, s, p)
    g = pow(n, s, p)
    r = e

    while True:
        t = b
        m = 0
        for m in range(r):
            if t == 1:
                break
            t = pow(t, 2, p)

        if m == 0:
            return x

        gs = pow(g, 2 ** (r - m - 1), p)
        g = (gs * gs) % p
        x = (x * gs) % p
        b = (b * g) % p
        r = m


def legendre_symbol(a, p):
    ls = pow(a, (p - 1) // 2, p)
    return -1 if ls == p - 1 else ls

In [3]:
M = 59
a, b = 1, 0

In [4]:
def makeEM(a, b, M):
    for x in range(0, M):
        y = modular_sqrt((x**3 + a*x + b)%M, M)
        if y > 0:
            yield x, y

In [5]:
elliptic_group = makeEM(0,1,47)

Вывод списка элементов эллиптической группы.

In [6]:
[*elliptic_group]

[(0, 1),
 (1, 7),
 (2, 3),
 (3, 34),
 (4, 21),
 (5, 28),
 (9, 42),
 (10, 25),
 (11, 4),
 (12, 32),
 (13, 6),
 (16, 14),
 (21, 12),
 (22, 36),
 (23, 18),
 (24, 17),
 (28, 2),
 (35, 24),
 (38, 27),
 (39, 37),
 (40, 9),
 (42, 8),
 (44, 16)]

#### Задание 3. Реализовать алгоритм обмена ключами для эллиптической группы EM(a,b)


Задание констант (Сообщение - "Hi, Alexashka", G - точка из группы EM(0, 1), q - порядок точки)

In [7]:
message = b'Hi, Alexashka'
G = np.array([2,3])
q = randprime(M*2, M*3)


In [8]:
def generate_key(G, M):
    secret_key = random.randint(M//2, M)
    public_key = secret_key*G
    return public_key, secret_key

In [9]:
def generate_common_key(self_secret_key, other_public_key):
    
    return self_secret_key * other_public_key

Генерация открытых и секретных ключей с обеих сторон и проверка равенства сгенерированного с обеих сторон общего ключа

In [10]:
a_public_key, a_secret_key = generate_key(G, M)
b_public_key, b_secret_key = generate_key(G, M)


assert generate_common_key(a_secret_key, b_public_key).all() == generate_common_key(b_secret_key, a_public_key).all(), 'Wrong common key'

#### Задание 4. Разработать алгоритм цифровой подписи на основе эллиптической группы EM(a, b)

In [263]:
def mod_inverse(a, m) : 
    m0 = m 
    y = 0
    x = 1
  
    if (m == 1) : 
        return 0 
    while (a > 1) : 
        q = a // m 
  
        t = m 
        m = a % m 
        a = t 
        t = y 
        y = x - q * y 
        x = t 

    if (x < 0) : 
        x = x + m0 
  
    return x 
  
  

In [264]:
def gen_digital_signature(G, M, q, message, a_public_key, a_secret_key):
    hash_message = int(hashlib.sha1(message).hexdigest(), 16)
    r = 0
    s = 0
    while r == 0 or s==0:
        k = randprime(1, q)
        kG = k*G
        r = kG[0]%q
        
        s = ((hash_message + a_secret_key)/k)%q
    return r, s

Генерация ЭЦП, представляющей собой элементы r и s

In [265]:
r, s = gen_digital_signature(G, M, q, message, a_public_key, a_secret_key)

In [266]:
r, s

(134, 88.0)

In [267]:
def check_digital_signature(G, M, q, message, r, s, a_public_key):
    #w = (1/s)%q
    w = mod_inverse(s, q)
    hash_message = int(hashlib.sha1(message).hexdigest(), 16)
    u1 = (hash_message*w) % q
    u2 = (r*w) % q 
    x1, y1 = u1*G + u2*a_public_key
    r1 = x1 % q
    print(w)
    print(r1)